### **Week 12 Knowledge Distilation 코드**

`net.py`의 `loss_fn_kd` 함수

In [ ]:
def loss_fn_kd(outputs, labels, teacher_outputs, params):
    """
    Knowledge Distillation 기반 KD loss 설명

    - student는 hard target뿐 아니라 teacher가 만든 확률분포 soft target을 함께 학습
    - 전체 loss = soft target(KL divergence) + hard target(Cross Entropy)의 가중합
    - temperature(T): softmax 분포를 부드럽게 만들어 teacher의 dark knowledge를 전달
    - alpha: teacher 모방(KD)과 정답 라벨 학습의 비중 조절
    - T^2: temperature로 인해 줄어든 gradient 크기를 보정
    """
    alpha = params.alpha              # KD(teacher 모방) 항과 정답(CE) 항을 섞는 비율
    T = params.temperature            # softmax 분포를 부드럽게 만드는 temperature

    KD_loss = nn.KLDivLoss()(
                # student의 softened log-probabilities
                F.log_softmax(outputs / T, dim=1),
                # teacher의 softened probabilities(soft targets)
                F.softmax(teacher_outputs / T, dim=1)
             # alpha로 가중 + temperature로 인한 gradient 축소를 T^2로 보정
             ) * (alpha * T * T) +

             # Standard supervised loss: 원래 정답 라벨에 대한 cross entropy
             F.cross_entropy(outputs, labels) * (1. - alpha)

    return KD_loss


`train.py`의 `train_kd` 함수

In [ ]:
def train_kd(model, teacher_model, optimizer, loss_fn_kd, dataloader, metrics, params):
    """
    Knowledge Distillation 기반 학습 루프

    - student(model)는 학습 모드, teacher_model은 고정된 추론 모드(eval)
    - 각 배치마다 student 출력과 teacher 출력(soft target)을 동시에 계산
    - KD loss를 사용해 teacher의 지식을 student로 전달하며 파라미터 업데이트
    """

    # student 모델은 학습 모드, teacher 모델은 평가 모드(gradient 없음)
    model.train()
    teacher_model.eval()

    # loss의 이동 평균 및 metric 저장용
    summ = []
    loss_avg = utils.RunningAverage()

    # 학습 진행 상황을 보기 위한 progress bar
    with tqdm(total=len(dataloader)) as t:
        for i, (train_batch, labels_batch) in enumerate(dataloader):

            # GPU 사용 시 데이터 이동
            if params.cuda:
                train_batch, labels_batch = train_batch.cuda(async=True), \
                                            labels_batch.cuda(async=True)

            # 입력과 라벨을 Variable로 변환
            train_batch, labels_batch = Variable(train_batch), Variable(labels_batch)

            # (1) student 모델 forward
            output_batch = model(train_batch)

            # (2) teacher 모델 forward (gradient 계산 없음)
            with torch.no_grad():
                output_teacher_batch = teacher_model(train_batch)

            if params.cuda:
                output_teacher_batch = output_teacher_batch.cuda(async=True)

            # (3) hard label + soft target을 함께 사용하는 KD loss 계산
            loss = loss_fn_kd(output_batch, labels_batch,
                              output_teacher_batch, params)

            # (4) 역전파 및 파라미터 업데이트
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # 일정 step마다 metric 계산
            if i % params.save_summary_steps == 0:
                output_batch = output_batch.data.cpu().numpy()
                labels_batch = labels_batch.data.cpu().numpy()

                summary_batch = {metric: metrics[metric](output_batch, labels_batch)
                                 for metric in metrics}
                summary_batch['loss'] = loss.data[0]
                summ.append(summary_batch)

            # 평균 loss 업데이트 및 출력
            loss_avg.update(loss.data[0])
            t.set_postfix(loss='{:05.3f}'.format(loss_avg()))
            t.update()

    # 전체 training step에 대한 metric 평균 계산
    metrics_mean = {metric: np.mean([x[metric] for x in summ]) for metric in summ[0]}
    metrics_string = " ; ".join("{}: {:05.3f}".format(k, v)
                                for k, v in metrics_mean.items())
    logging.info("- Train metrics: " + metrics_string)